# Predicting painting authors

In this notebook, I will build a model to predict authors of paintings. I will use the [dataset](https://www.kaggle.com/datasets/ikarus777/best-artworks-of-all-time) from Kaggle that has 8,000+ paintings by 50 most famous artists to train the model.

Let's start with importing the necessary libraries and exploring the information about the paintings in the dataset.

In [2]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
image_size = (256, 256)
batch_size = 32

train_ds = tf.keras.utils.image_dataset_from_directory(
    'data\images',
    validation_split=0.2,
    subset='training',
    seed=17,
    image_size=image_size,
    batch_size=batch_size,
)

valid_ds = tf.keras.utils.image_dataset_from_directory(
    'data\images',
    validation_split=0.2,
    subset='validation',
    seed=17,
    image_size=image_size,
    batch_size=batch_size,
)

Found 8446 files belonging to 50 classes.
Using 6757 files for training.
Found 8446 files belonging to 50 classes.
Using 1689 files for validation.


In [3]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
valid_ds = valid_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(50)
])

In [ ]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
model.fit(
  train_ds,
  validation_data=valid_ds,
  epochs=10
)

In [ ]:
base_model = tf.keras.applications.vgg16.VGG16(input_shape = (256, 256, 3), include_top = False, weights = 'imagenet')
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(50)

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [4]:
base_model = tf.keras.applications.ResNet50(input_shape=(256, 256,3), include_top=False, weights="imagenet")
base_model.trainable = False

global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(50)

model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

94765736/94765736 [==============================] - 556s 6us/step


In [5]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [6]:
model.fit(
  train_ds,
  validation_data=valid_ds,
  epochs=10
)

Epoch 1/10
212/212 [==============================] - 594s 3s/step - loss: 2.1536 - accuracy: 0.4398 - val_loss: 1.5746 - val_accuracy: 0.5678
Epoch 2/10
212/212 [==============================] - 565s 3s/step - loss: 1.0996 - accuracy: 0.6916 - val_loss: 1.3613 - val_accuracy: 0.6157
Epoch 3/10
212/212 [==============================] - 586s 3s/step - loss: 0.7720 - accuracy: 0.7913 - val_loss: 1.2508 - val_accuracy: 0.6501
Epoch 4/10
212/212 [==============================] - 605s 3s/step - loss: 0.5741 - accuracy: 0.8548 - val_loss: 1.1933 - val_accuracy: 0.6684
Epoch 5/10
212/212 [==============================] - 581s 3s/step - loss: 0.4401 - accuracy: 0.9023 - val_loss: 1.1759 - val_accuracy: 0.6838
Epoch 6/10
212/212 [==============================] - 580s 3s/step - loss: 0.3451 - accuracy: 0.9315 - val_loss: 1.1811 - val_accuracy: 0.6939
Epoch 7/10
212/212 [==============================] - 576s 3s/step - loss: 0.2755 - accuracy: 0.9523 - val_loss: 1.1899 - val_accuracy: 0.6868